In [26]:
import pandas as pd
import numpy as np
import gzip
import pickle
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def duplicate( array_input, size ):
  vg_input = np.empty([size, 48, 48, 3])
  for index, item in enumerate(vg_input):
      item[:, :,0] = array_input[index]
      item[:, :,1] = array_input[index]
      item[:, :,2] = array_input[index]
  return vg_input

In [37]:
from keras.utils import np_utils
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense,Dropout,Embedding
from keras.optimizers import SGD, adam

X_train, y_train = list(),list()
#raw_training = raw_data[raw_data["Usage"] == "Training"]

f = gzip.open("/content/drive/My Drive/Colab Notebooks/TPFinal/X_train-5.pkl.gz", 'rb')
varxtrain = pickle.load(f)
f.close()

f = gzip.open("/content/drive/My Drive/Colab Notebooks/TPFinal/y_train.pkl.gz", 'rb')
y_train = pickle.load(f)
f.close()

  
X_train = np.array(varxtrain, dtype = 'float32')
y_train = np.array( y_train, dtype='int16' )

print(X_train.shape)
print(y_train.shape)

(1308, 48, 48, 3)
(1308, 7)


In [38]:
from google.colab import drive
import cv2
from keras.utils import np_utils
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense,Dropout,Embedding
from keras.optimizers import SGD, adam

drive.mount('/content/drive')

vgg16 = VGG16(include_top=False, input_shape=(48, 48, 3), weights='imagenet')

def get_vgg16_output(vgg16, array_input, n_feature_maps):
  #vg_input = duplicate(array_input, n_feature_maps)
  vg_input = array_input
  picture_train_features = vgg16.predict(vg_input)
  print(picture_train_features.shape)
  del (vg_input)
  feature_map = np.empty([n_feature_maps, 512])
  print(feature_map)
  for idx_pic, picture in enumerate(picture_train_features):
      feature_map[idx_pic] = picture
  return feature_map

#x_train_feature_map = get_vgg16_output(vgg16, x_train_matrix, len(x_train_matrix) )
#print(x_train_feature_map.shape)
x_train_feature_map = get_vgg16_output(vgg16, varxtrain, len(varxtrain) )
print(x_train_feature_map.shape)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(1308, 1, 1, 512)
[[-1.16636551e-17  5.63768808e-25  2.07226185e-22 ...  6.59339149e-23
  -8.47856889e-20  9.06903138e-23]
 [-4.32131918e-19  2.35307033e-18  1.05973494e-14 ...  5.11404808e-17
   3.65621282e-28 -8.02694264e-22]
 [-1.70866039e-23  9.06674105e-19 -3.29401502e-22 ... -3.67847930e-16
  -7.01866447e-17 -3.73830144e-30]
 ...
 [-3.48891625e-18 -8.18826242e-18 -1.97256280e-21 ... -1.09796221e-20
  -1.38677268e-19 -2.53529499e-17]
 [-5.07182976e-22 -9.89157259e-17  2.42145522e-20 ... -8.17518215e-39
  -5.72296453e-21  3.42083052e-17]
 [ 1.04093135e-16 -2.09147071e-16 -2.97344510e-24 ...  7.25250941e-18
  -7.43968271e-22  8.47075851e-19]]
(1308, 512)


In [39]:
import gzip
import pickle

X_test, y_test = list(),list()
#raw_training = raw_data[raw_data["Usage"] == "PublicTest"]

f = gzip.open("/content/drive/My Drive/Colab Notebooks/TPFinal/X_test-5.pkl.gz", 'rb')
varxtest = pickle.load(f)
f.close()

f = gzip.open("/content/drive/My Drive/Colab Notebooks/TPFinal/y_test.pkl.gz", 'rb')
y_test = pickle.load(f)
f.close()
  
X_test = np.array(varxtest, dtype = 'float32')
#y_test = np.array(np_utils.to_categorical(y_test), dtype = 'int16')
y_test = np.array( y_test, dtype='int16' )

print(X_test.shape)
print(y_test.shape)

x_test_feature_map = get_vgg16_output(vgg16, varxtest, len(varxtest) )
print(x_test_feature_map.shape)

(327, 48, 48, 3)
(327, 7)
(327, 1, 1, 512)
[[0.00000000e+000 5.20185816e-315 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 5.15926664e-315]
 [0.00000000e+000 5.21533256e-315 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 5.14398796e-315]
 [5.29689143e-315 5.22807239e-315 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 5.12754477e-315]
 ...
 [5.27032442e-315 5.22815027e-315 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [5.27458563e-315 5.23103691e-315 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [5.19536423e-315 5.21635006e-315 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 5.14518784e-315]]
(327, 512)


In [40]:
#Modelo

topLayerModel = Sequential()
topLayerModel.add(Dense(256, input_shape=(512,), activation='relu'))
topLayerModel.add(Dense(256, input_shape=(256,), activation='relu'))
topLayerModel.add(Dropout(0.5))
topLayerModel.add(Dense(128, input_shape=(256,), activation='relu'))
topLayerModel.add(Dense(7, activation='softmax'))

adamax = adam()
topLayerModel.compile(loss='categorical_crossentropy',optimizer=adamax, metrics=['accuracy'])
topLayerModel.fit(x_train_feature_map, y_train, validation_data=(x_train_feature_map, y_train),epochs=35, batch_size= 10) 
#topLayerModel.fit(varxtrain, y_train, validation_data=(x_, y_train),nb_epoch=30, batch_size= 10) 


W0621 18:41:08.912271 140282773219200 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1308 samples, validate on 1308 samples
Epoch 1/35
1308/1308 [==============================] - 2s 1ms/step - loss: 1.8054 - acc: 0.2706 - val_loss: 1.6147 - val_acc: 0.3823
Epoch 2/35
1308/1308 [==============================] - 1s 531us/step - loss: 1.5934 - acc: 0.3570 - val_loss: 1.3919 - val_acc: 0.4320
Epoch 3/35
1308/1308 [==============================] - 1s 541us/step - loss: 1.3890 - acc: 0.4633 - val_loss: 1.1535 - val_acc: 0.5642
Epoch 4/35
1308/1308 [==============================] - 1s 539us/step - loss: 1.1861 - acc: 0.5313 - val_loss: 0.9690 - val_acc: 0.6162
Epoch 5/35
1308/1308 [==============================] - 1s 525us/step - loss: 1.0758 - acc: 0.5879 - val_loss: 0.8396 - val_acc: 0.6797
Epoch 6/35
1308/1308 [==============================] - 1s 540us/step - loss: 0.9331 - acc: 0.6246 - val_loss: 0.7571 - val_acc: 0.7278
Epoch 7/35
1308/1308 [==============================] - 1s 515us/step - loss: 0.7530 - acc: 0.7080 - val_loss: 0.5667 - val_acc: 0.7928
Ep

In [42]:
scores = topLayerModel.evaluate( x_test_feature_map,y_test)
print("\n%s: %.2f%%" %( topLayerModel.metrics_names[1], scores[1] * 100 ) )
print(topLayerModel.metrics_names)
print(scores)


327/327 [==============================] - 0s 93us/step

acc: 95.72%
['loss', 'acc']
[0.08652139826469829, 0.9571865445247848]


In [43]:
y_pred = topLayerModel.predict_classes(x_test_feature_map)
#print(topLayerModel.predict_proba(x_test_feature_map[:2])*100)
#print(np.round(topLayerModel.predict(x_test_feature_map[:2]),3))
print(x_test_feature_map.shape)
#print(y_pred[1])

y_classes = [np.argmax(y, axis=None, out=None) for y in y_test] #Nos permite volver a ver el valor numerico y no categorico

numero_columnas = numero_filas = 7
matriz = [[0] * numero_columnas for i in range(numero_filas)]
matriz = np.array(matriz)
for k in range( len(y_pred) ):
  matriz[int(y_classes[k])][int(y_pred[k])] += 1 #fila valor real columna prediccion
print(matriz)
print ("Precision: " + str( float ( np.trace(np.array(matriz)) ) / len(y_pred) ))


(327, 512)
[[39  0  3  1  0  2  0]
 [ 0 18  0  0  0  0  0]
 [ 0  0 58  0  0  0  1]
 [ 0  0  0 24  0  0  1]
 [ 0  0  0  0 69  0  0]
 [ 0  0  3  0  0 24  1]
 [ 1  0  1  0  0  0 81]]
Precision: 0.9571865443425076


In [0]:
!pip install h5py

In [0]:
from keras.models import model_from_json
model_json = topLayerModel.to_json()
with open("model.json", "w") as json_file:
  json_file.write(model_json)
#serializan los pesos (weights) para HDF5
topLayerModel.save_weights("model.h5")
print("Modelo guardado en el PC")
# despues...
# carga el json y crea el modelo
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# se cargan los pesos (weights) en el nuevo modelo
loaded_model.load_weights("model.h5")
print("Modelo cargado desde el PC")
# se evalua el modelo cargado con los datos de los test
loaded_model.compile(loss='categorical_crossentropy',optimizer=adamax, metrics=['accuracy'])

score = loaded_model.evaluate(x_test_feature_map, y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Modelo guardado en el PC
Modelo cargado desde el PC
acc: 96.02%


In [0]:
pred = loaded_model.predict_classes(x_test_feature_map)
numero_columnas = numero_filas = 7
confusion = [[0] * numero_columnas for i in range(numero_filas)]
confusion = np.array(confusion)
for k in range( len(y_pred) ):
  confusion[int(y_classes[k])][int(pred[k])] += 1 #fila valor real columna prediccion
print(confusion)
print ("Precision: " + str( float ( np.trace(np.array(confusion)) ) / len(pred) ))


[[43  0  1  0  0  0  1]
 [ 0 18  0  0  0  0  0]
 [ 0  0 55  0  0  0  4]
 [ 0  0  0 23  2  0  0]
 [ 0  0  0  0 67  0  2]
 [ 0  0  0  0  0 26  2]
 [ 1  0  0  0  0  0 82]]
Precision: 0.9602446483180428
